In [ ]:
import pandas as pd

In [ ]:
dataset_train1 = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', index_col='id')
dataset_test1 = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv', index_col='id')
y = dataset_train1.target
y = pd.DataFrame(y)
dataset_train = dataset_train1.drop(['target'], axis=1)

reduce memory usage

In [ ]:
for col in dataset_train.columns:
    if dataset_train[col].dtype == 'float64':
        dataset_train[col] = dataset_train[col].astype('float32')
        dataset_test1[col] = dataset_test1[col].astype('float32')
    if dataset_train[col].dtype == 'int64':
        dataset_train[col] = dataset_train[col].astype('int8')
        dataset_test1[col] = dataset_test1[col].astype('int8')

In [ ]:
dataset_train['ones_count'] = dataset_train[dataset_train == 1].sum(axis=1)

In [ ]:
dataset_test1['ones_count'] = dataset_test1[dataset_test1 == 1].sum(axis=1)

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, PowerTransformer

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(dataset_train)
dataset_test_sc = x_scaler.transform(dataset_test1)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
with strategy.scope():
    model = load_model('../input/tps1021/best_85294.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
output = pd.DataFrame({'Id': dataset_test1.index,'target': preds[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 